In [2]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import re

In [73]:
folder_path = './mof_features'
df_fea = pd.read_csv(os.path.join(folder_path, 'Features_RACS.csv'), index_col=0)
# duplicate_index = df_fea.index.duplicated()
# df_no_duplicates = df_fea[~duplicate_index]
# df_no_duplicates.to_csv(os.path.join(folder_path, 'matminer_racs_all.csv'))
df_fea.shape

(8546, 143)

In [22]:
# folder_path = './features'

# # Get all feature csv in the folder
# all_files = os.listdir(folder_path)
# filtered_files = [file for file in all_files if file.startswith('matminer_racs_')]
# fea_lst = []
# for file in filtered_files:
#     df_fea = pd.read_csv(os.path.join(folder_path, file), index_col=0)
#     fea_lst.append(df_fea)
# df_all = pd.concat(fea_lst)

# df_all.to_csv(os.path.join(folder_path, 'matminer_racs_all.csv'))


/tmp/ipykernel_224572/2635449414.py:8: DtypeWarning: Columns (111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fea = pd.read_csv(os.path.join(folder_path, file), index_col=0)
/tmp/ipykernel_224572/2635449414.py:8: DtypeWarning: Columns (111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fea = pd.read_csv(os.path.join(folder_path, file), index_col=0)


In [97]:
tl = False
folder_path = './mof_features'
df_fea = pd.read_csv(os.path.join(folder_path, 'Features_RACS.csv'), index_col=0)

# df_fea.values
label_encoder = LabelEncoder()
# if 'compound possible' in df_fea.columns:
#     df_fea['compound possible'] = df_fea['compound possible'].replace({'False': 0, 'True': 1, False: 0, True: 1, '0.0': 0}).astype(float)
# for str_label in ['HOMO_character', 'HOMO_element', 'LUMO_character', 'LUMO_element']:
#     if str_label in df_fea.columns:
#         df_fea[str_label] = label_encoder.fit_transform(df_fea[str_label].astype(str))
for str_label in ['Metal']:
    if str_label in df_fea.columns:
        df_fea[str_label] = label_encoder.fit_transform(df_fea[str_label].astype(str))
imputer = SimpleImputer(strategy='mean')
feature_cols = df_fea.columns[2:]
df_fea[feature_cols] = imputer.fit_transform(df_fea[feature_cols].values)
df_fea['sample'] = df_fea['MOF Name'] + '.cif'
df_fea.to_csv(os.path.join(folder_path, 'racs_all_clean.csv'))
# df_fea[['HOMO_character', 'HOMO_element', 'LUMO_character', 'LUMO_element']]


/tmp/ipykernel_187314/2892756937.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fea['sample'] = df_fea['MOF Name'] + '.cif'


### Combine with ALIGNN Embeddings (Optional)


In [91]:
tl = True
run = 'mof_dband_embed'
embed_filepath = f"/scratch/yll6162/ALIGNNTL/examples/{run}/x+y+z/data0.csv"
df_embed = pd.read_csv(embed_filepath)
df_embed = df_embed.drop_duplicates(subset = ['id'])
df_fea_all = df_fea.merge(df_embed, how='left', left_on = "MOF Name", right_on = "id").drop(columns = ['oxo','id'])
df_fea = df_fea_all


### Feature Scale

In [98]:
from sklearn.preprocessing import StandardScaler

racs_col = [col for col in df_fea.columns if col.startswith("racs_")]
embed_cols = [str(i) for i in range(768)]
scaler_A = StandardScaler()
df_fea[racs_col] = scaler_A.fit_transform(df_fea[racs_col].values)
# if embed_cols:
#     scaler_B = StandardScaler()
#     df_fea[embed_cols] = scaler_B.fit_transform(df_fea[embed_cols].values)

### Generate Train Dataset

In [99]:
prop = 'Oxo Formation Energy'
output_dir = './data/'
df_qmof = pd.read_csv("./labels/qmofruns_2_suffled.csv", index_col = 0)
df_qmof['sample'] = df_qmof.MOF + '.cif'

df_qmof['prop'] = df_qmof[prop]
df_qmof = df_qmof[['sample', 'Site', prop]]

df_fea['sample'] = df_fea['MOF Name'] + '.cif'

df_sample = df_qmof.merge(df_fea, how='left', left_on=['sample', 'Site'], right_on = ['sample','Metal_index'])
df_sample['ids'] = df_sample['sample'] + '_' + df_sample['Site'].astype(str)
df_sample = df_sample.drop(columns = ['sample', 'Site', 'MOF Name', 'Metal_index', 'Metal'])
df_sample = df_sample.assign(**{prop: df_sample.pop(prop)})
df_sample = df_sample.assign(**{'ids': df_sample.pop('ids')})
if tl:
    df_sample.to_csv(os.path.join(output_dir, f"full_data_{run}_{prop}.csv"), index=False)
else:
    df_sample.to_csv(os.path.join(output_dir, f"full_data_racs_{prop}.csv"), index=False)

/tmp/ipykernel_187314/3346178874.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sample['ids'] = df_sample['sample'] + '_' + df_sample['Site'].astype(str)


In [94]:
df_sample.describe()

,racs_bb-linker_all_prop-I_scope-1_propagg-diff_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-I_scope-1_propagg-product_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-I_scope-2_propagg-diff_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-I_scope-2_propagg-product_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-I_scope-3_propagg-diff_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-I_scope-3_propagg-product_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-T_scope-1_propagg-diff_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-T_scope-1_propagg-product_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-T_scope-2_propagg-diff_corragg-sum_bbagg-sum,racs_bb-linker_all_prop-T_scope-2_propagg-product_corragg-sum_bbagg-sum,...,759,760,761,762,763,764,765,766,767,Oxo Formation Energy
count,435.0,435.000000,435.0,435.000000,435.0,435.000000,435.000000,435.000000,435.000000,435.000000,...,4.350000e+02,4.350000e+02,4.350000e+02,4.350000e+02,4.350000e+02,4.350000e+02,4.350000e+02,4.350000e+02,4.350000e+02,438.000000
mean,0.0,-0.127581,0.0,-0.098059,0.0,-0.102273,-0.059484,-0.107261,0.014249,0.020795,...,6.061425e-01,-1.824399e-01,-5.615201e-02,2.934498e+00,6.092349e-01,-7.398330e-01,-4.064582e-01,2.336539e-01,-1.571390e-01,0.423311
std,0.0,0.881061,0.0,0.939159,0.0,0.918839,1.023276,0.969865,1.452872,1.676241,...,7.780509e-16,1.361589e-15,5.349100e-16,3.112204e-15,2.889904e-15,2.445303e-15,2.334153e-15,5.835382e-16,6.669008e-16,1.239723
min,0.0,-1.427075,0.0,-1.487900,0.0,-1.417240,-1.392534,-1.410062,-0.889733,-0.788547,...,6.061425e-01,-1.824399e-01,-5.615201e-02,2.934498e+00,6.092349e-01,-7.398330e-01,-4.064582e-01,2.336539e-01,-1.571390e-01,-2.730356
25%,0.0,-0.696386,0.0,-0.705605,0.0,-0.704811,-0.699877,-0.714068,-0.528073,-0.468874,...,6.061425e-01,-1.824399e-01,-5.615201e-02,2.934498e+00,6.092349e-01,-7.398330e-01,-4.064582e-01,2.336539e-01,-1.571390e-01,-0.462636
50%,0.0,-0.350270,0.0,-0.307796,0.0,-0.334636,-0.325162,-0.345443,-0.257692,-0.231123,...,6.061425e-01,-1.824399e-01,-5.615201e-02,2.934498e+00,6.092349e-01,-7.398330e-01,-4.064582e-01,2.336539e-01,-1.571390e-01,0.543632
75%,0.0,0.209109,0.0,0.168053,0.0,0.194957,0.288010,0.154041,0.038274,0.016449,...,6.061425e-01,-1.824399e-01,-5.615201e-02,2.934498e+00,6.092349e-01,-7.398330e-01,-4.064582e-01,2.336539e-01,-1.571390e-01,1.459677
max,0.0,4.250622,0.0,4.283190,0.0,4.289397,6.067719,7.628553,24.118057,30.404196,...,6.061425e-01,-1.824399e-01,-5.615201e-02,2.934498e+00,6.092349e-01,-7.398330e-01,-4.064582e-01,2.336539e-01,-1.571390e-01,3.213310
